In [20]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from spai.data.satellite.utils import get_box_or_gdfs_by_place_name
locations = get_box_or_gdfs_by_place_name("begues")  # returns a list of dicts like this: [ {'name': 'Fraga, Catalunya', 'gdf': <gpd.GeoDataFrame object>}, ...]
locations[0]["gdf"]  # returns a GeoDataFrame of the Bounding Box of the location

,geometry
0,"POLYGON ((1.96131 41.27823, 1.96131 41.35582, ..."


In [10]:
import ipywidgets
import ipyleaflet
from ipyleaflet import Map, basemaps, basemap_to_tiles
from geopandas import GeoSeries

# Create map
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(41.46183613708533, 2.058000718868186),
    zoom=5
    )

# Handle click on map
selected_geodata = None
def handle_click(event, feature, id, properties):
    global selected_geodata
    selected_geodata = feature
    print(feature)

# Add gdfs to map
for idx, location in enumerate(locations):
    geodata = ipyleaflet.GeoData(
        geo_dataframe=location["gdf"],
        style={
            "color": "blue",
            "opacity": 1,
            "weight": 1.9,
            "fillOpacity": 0.5,
        },
        hover_style={"fillColor": "#b08a3e", "fillOpacity": 0.9},
    )
    geodata.on_click(handle_click)
    m.add_layer(geodata)
    
    # Reproject the geometries to a projected CRS
    projected_gdf = location["gdf"].to_crs(epsg=3857)  # '3857': Desired projected CRS

    # Compute the centroid on the reprojected geometries
    centroid = projected_gdf["geometry"].centroid
    
    # Transform the centroid coordinates back to geographic CRS
    centroid = centroid.to_crs(epsg=4326)  # '4326': original geographic CRS
    
    # Add a marker with location name for each location
    marker = ipyleaflet.Marker(location=(centroid.y[0], centroid.x[0]), title=location["name"] if location["name"] else "Unknown")
    m.add_layer(marker)
m

Map(center=[41.46183613708533, 2.058000718868186], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [32]:
selected_geodata

{'id': '0',
 'type': 'Feature',
 'properties': {},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-3.0529852, 39.8846264],
    [-3.0529852, 41.1657381],
    [-4.5790058, 41.1657381],
    [-4.5790058, 39.8846264],
    [-3.0529852, 39.8846264]]]}}